In [ ]:
#importing data
import pandas as pd
file_path = 'Credit Banking_Project - 1.xls' 
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

In [ ]:
#creating data frames for each sheet
df_acquisition = pd.read_excel(xls, 'Customer Acqusition')
df_Spend =pd.read_excel(xls,'Spend')
df_Repayment=pd.read_excel(xls,'Repayment')

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
sns.histplot(df_acquisition['Age'],kde=True , bins=20,color='red', edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
#treating age column
df_adults= df_acquisition[df_acquisition['Age']>=18]
print(df_adults)
df_minors = df_acquisition[df_acquisition['Age']<18]
print(df_minors)
mean_adult_age = df_adults['Age'].mean()
print(f"Mean age of adults: {mean_adult_age}")
df_minors['Age'] = mean_adult_age
df_acquisition.loc[df_acquisition['Age']<18,'Age']=mean_adult_age
print(df_acquisition['Age'].mean())

In [ ]:
#montly spend of each customer
df_Spend['Month']=pd.to_datetime(df_Spend['Month'],format ='%d-%b-%y')
df_Spend['day']=df_Spend['Month'].dt.day
df_Spend['month']=df_Spend['Month'].dt.month
df_Spend['year']=df_Spend['Month'].dt.year
grouped = df_Spend.groupby(['Costomer', 'year', 'month'])['Amount'].sum().reset_index()
# grouped_df = grouped.apply(lambda x: x.reset_index(drop=True)).reset_index(drop=True)
print(grouped[['Costomer', 'year', 'month', 'Amount']])
#People in which segment are spending more money.
total_spent = grouped.groupby('Costomer')['Amount'].sum().reset_index()
top_1_customer = total_spent.sort_values(by='Amount', ascending=False).head(1)
df_acquisition = df_acquisition.rename(columns={'Customer': 'Costomer'})
merged_df = top_1_customer.merge(df_acquisition[['Costomer', 'Segment']], on='Costomer', how='left')
merged_df

In [ ]:
#merging with Limit
df_acquisition = df_acquisition.rename(columns={'Customer': 'Costomer'})
merged_df = grouped.merge(df_acquisition[['Costomer', 'Limit']], on='Costomer', how='left')
print(merged_df)
#Comparing limit and monthly spent of customer
print("customer who have spent more than his/her Credit Limit for any particular month")
over_limit = merged_df[merged_df['Amount'] > merged_df['Limit']]
customers_over_limit = over_limit['Costomer'].unique()
customers_over_limit

In [ ]:
#Monthly Repayment of each Costomer 
df_Repayment['month']=df_Repayment['Month'].dt.month
df_Repayment['year']=df_Repayment['Month'].dt.year
grouped=df_Repayment.groupby(['Costomer','month','year'])['Amount'].sum().reset_index()
print(grouped[['Costomer','year','month','Amount']])
#HIghest paying 10 customers
total_repayment = grouped.groupby('Costomer')['Amount'].sum().reset_index()
top_10_customers = total_repayment.sort_values(by='Amount', ascending=False).head(10)
top_10_customers

In [ ]:
#in which age group people are spending more money
df_acquisition = df_acquisition.rename(columns={'Customer': 'Costomer'})
merged_df = pd.merge(df_acquisition, df_Spend, left_on='Costomer', right_on='Costomer', how='inner')

merged_df['age_group'] = pd.cut(merged_df['Age'],
                                 bins=[-float('inf'), 18, 30, 40, float('inf')],
                                 labels=['Under 18', '18-29', '30-39', '40 and above'],
                                 right=False)
merged_df[['Costomer','age_group','Amount']]
grouped = merged_df.groupby('age_group')['Amount'].sum().reset_index()
grouped = grouped.sort_values(by='Amount', ascending=False).reset_index(drop=True)
value = grouped.loc[0, 'age_group']
print("the highest spending age group is---- ", value)

In [ ]:
spend_merged = pd.merge(df_acquisition[['Costomer', 'Segment']], df_Spend, on='Costomer', how='inner')
repay_merged = pd.merge(df_acquisition[['Costomer', 'Segment']], df_Repayment, on='Costomer', how='inner')

spend_by_segment = spend_merged.groupby('Segment')['Amount'].sum().reset_index(name='Total_Spend')
repay_by_segment = repay_merged.groupby('Segment')['Amount'].sum().reset_index(name='Total_Repayment')

profit_df = pd.merge(spend_by_segment, repay_by_segment, on='Segment')
profit_df['Profit'] = profit_df['Total_Repayment'] - profit_df['Total_Spend']
most_profitable = profit_df.sort_values(by='Profit', ascending=False).reset_index(drop=True)
print(most_profitable)
value=most_profitable.loc[0,'Segment']
print("the mose profitable segment is----",value)

In [ ]:
#In which category the customers are spending more money?
grouped=df_Spend.groupby(['Type'])['Amount'].sum().reset_index()
most_spend_category=grouped.sort_values(by='Amount',ascending=False).reset_index(drop=True)
most_spend_category
value=most_spend_category.loc[0,'Type']
print("the most spending Category is----",value)

In [ ]:
#Impose an interest rate of 2.9% for each customer for any due amount.
merged_df = pd.merge(df_Spend, df_Repayment, on='Costomer', how='inner')
merged_df['Month_x'] = pd.to_datetime(merged_df['Month_x'], format='%d-%b-%y')
merged_df['monthly'] = merged_df['Month_x'].dt.month
merged_df['year'] = merged_df['Month_x'].dt.year

monthly_spend = merged_df.groupby(['Costomer', 'year', 'monthly'])['Amount_x'].sum().reset_index(name='monthly_spend')
monthly_repayment = merged_df.groupby(['Costomer', 'year', 'monthly'])['Amount_y'].sum().reset_index(name='monthly_repayment')

merged_monthly = pd.merge(monthly_spend, monthly_repayment, on=['Costomer', 'year', 'monthly'], how='inner')

merged_monthly['due_amount'] = merged_monthly['monthly_spend'] - merged_monthly['monthly_repayment']
merged_monthly['due_amount'] = merged_monthly['due_amount'].apply(lambda x: x if x > 0 else 0)
merged_monthly['interest'] = merged_monthly['due_amount'] * 0.029

print(merged_monthly[['Costomer', 'year', 'monthly', 'due_amount', 'interest']])


In [ ]:
merged_df = df_Spend.merge(df_acquisition, left_on='Costomer', right_on='Costomer')
merged_df['monthly'] = pd.to_datetime(merged_df['Month']).dt.month

grouped = merged_df.groupby(['monthly', 'Limit'], as_index=False).agg(monthly_spend=('Amount', 'sum'))

grouped['bank_profit'] = grouped.apply(
    lambda row: row['Limit'] * 0.02 if row['monthly_spend'] > row['Limit'] else 0,
    axis=1
)

print(grouped)